In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import random
import numpy as np
import os

from barbar import Bar
%matplotlib inline

In [2]:
EPOCH = 30
C = 100
BATCH = 1024
PATH = 'D:/文件/作业/神经网络/期末pj/resnet34-cutout'

writer = SummaryWriter(log_dir=PATH)

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 2021
seed_everything(SEED)

In [4]:
def cutout(batch):
    input, target = batch

    input_s = torch.zeros_like(input)
    lamb = np.random.uniform(0.0, 1.0)

    H, W = input.shape[2:]
    r_x = np.random.uniform(0, W)
    r_y = np.random.uniform(0, H)
    r_w = W * np.sqrt(1 - lamb)
    r_h = H * np.sqrt(1 - lamb)
    x1 = int(np.round(max(r_x - r_w / 2, 0)))
    x2 = int(np.round(min(r_x + r_w / 2, W)))
    y1 = int(np.round(max(r_y - r_h / 2, 0)))
    y2 = int(np.round(min(r_y + r_h / 2, H)))

    input[:, :, x1:x2, y1:y2] = input_s[:, :, x1:x2, y1:y2]

    return input, target

In [5]:
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH,
                                          shuffle=True, num_workers=2)

test_transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
net = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=False).cuda()

Using cache found in C:\Users\li/.cache\torch\hub\pytorch_vision_v0.6.0


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [8]:
for epoch in range(EPOCH):
    
    ################
    ### Training ###
    ################
    running_loss = 0.0
    for i, data in enumerate(Bar(trainloader)):
        data = cutout(data)
        inputs, labels = data[0].cuda(), data[1]

        optimizer.zero_grad()

        outputs = net(inputs)

        target = labels.cuda()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d] loss: %.3f' %
          (epoch + 1, running_loss / 50000 * i))
    writer.add_scalar("loss", running_loss / 50000 * i, epoch)
    running_loss = 0.0


    ################
    ### Testing  ###
    ################
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].cuda(), data[1].cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
    writer.add_scalar("test_accuracy", 100 * correct / total, epoch)

    torch.save(net.state_dict(), PATH + '/model_latest.h5')
    scheduler.step()

50000/50000: [===============================>] - ETA 9.3s
[1] loss: 0.229
Accuracy of the network on the 10000 test images: 9 %
50000/50000: [===============================>] - ETA 1.4s
[2] loss: 0.185
Accuracy of the network on the 10000 test images: 13 %
50000/50000: [===============================>] - ETA 1.4s
[3] loss: 0.176
Accuracy of the network on the 10000 test images: 17 %
50000/50000: [===============================>] - ETA 1.4s
[4] loss: 0.168
Accuracy of the network on the 10000 test images: 20 %
50000/50000: [===============================>] - ETA 1.4s
[5] loss: 0.158
Accuracy of the network on the 10000 test images: 22 %
50000/50000: [===============================>] - ETA 1.5s
[6] loss: 0.152
Accuracy of the network on the 10000 test images: 24 %
50000/50000: [===============================>] - ETA 1.3s
[7] loss: 0.151
Accuracy of the network on the 10000 test images: 25 %
50000/50000: [===============================>] - ETA 1.4s
[8] loss: 0.145
Accuracy of the 

In [9]:
torch.save(net.state_dict(), PATH + '/resnet34-model.h5')

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].cuda(), data[1].cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 37.970 %
